

1. **데이터셋 준비 및 전처리** : ``torchvision``을 사용하여 CIFAR10 training, test 데이터셋을 load하고 normalize
2. **분류기 모델 설계** : CNN을 사용하여 분류기 모델 설계
3. **Loss function 정의** : 학습 시 사용되는 loss function 정의하기
4. **모델 학습하기** : 1.의 training 데이터셋으로 설정한 epoch만큼 모델을 학습
5. **테스트하기** : 학습이 완료된 모델을 1.의 test 데이터셋으로 테스트하기


**torch**는 pytorch의 가장 상위에 있는 package로써 라이브러리에서 이것저것 불러오기 위해 import

**torchvision**은 Computer vision에서 사용하는 각종 테크닉들을 torch와 연동하여 구현한 라이브러리

In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

🔴🟠🟡🟢🔵🟣

> 궁금증
```
[0,1]에서 [-1,1]로 normalize하기
(0-0.5)/0.5 = -1
(1-0.5)/0.5 = 1
[참고] transforms.normalize (https://pytorch.org/vision/stable/transforms.html)
```
> 문제점
```
데이터 시각화하는 부분에서 명세서(4개)와 달리 128개의 이미지가 출력됨.
trainloader에 128개의 이미지가 담겨있었기 때문. 이것을 4개로 조정해주면 된다.
근데 batch size를 작게하는 것이 학습에도 좋을지는 모르겠다.
```
> 궁금증 (질문하기)
```
input 데이터의 size를 찍어보면 ... images.size()
[128,3,32,32]가 나온다.
각각 [batch size, channel, 가로, 세로]이다.
데이터 시각화하기 위햇 [가로, 세로, channel] 형태로 변환해야 하는데
visualize 함수에서는 (1,2,0)으로 transform한다. 이건 뭘까..
```

## 1. 데이터셋 준비 및 전처리
- ``torchvision``을 사용하여 CIFAR10 을 load 할 수 있습니다.
- ``torchvision.transform`` 을 사용하여 데이터를 normalize할 수 있습니다.

**references**\
dataloader(pytorch doc)\
https://pytorch.org/docs/stable/data.html

dataloader parameter(blog)\
https://subinium.github.io/pytorch-dataloader/


In [11]:
# [0,1] 범위의 데이터셋을 [-1, 1] 범위의 값으로 normalize하도록 transform 정의
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

# torchvision으로 CIFAR10 trainset load, trainset dataloader 정의
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# torchvision으로 CIFAR10 testset load, trainset dataloader 정의
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
# CIFAR10의 10개의 class 정의
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### 데이터 시각화
- ``matplotlib``는 python visualization library
- ``matplotlib``의 imshow 함수를 데이터들을 시각화 할 수 있습니다.


**references**

matplotlib doc\
https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html


In [12]:
# Req. 1-1	데이터셋 준비 및 전처리
# 이미지를 시각화하는 함수
def visualize(img):
    # [-1, 1] 범위로 normalize된 데이터를 [0,1] 범위로 unnormalize
    img = img / 2 + 0.5

    # img를 numpy값으로 변환
    np_img = img.numpy()

    # plt.imshow함수로 시각화
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()

# 트레이닝 데이터를 랜덤 샘플
dataiter = iter(trainloader)
images, labels = dataiter.next() ## image

# show images
visualize(torchvision.utils.make_grid(images))

# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

 deer  bird plane  bird


🔴🟠🟡🟢🔵🟣

> 용어
```
채널이란, 색 성분.
흑백이면 1-채널, RGB이면 3-채널이다
```
```
super()은 태초의 할머니까지 가지만
super(인자)는 인자까지만 탐색한다
[참고] super() vs super(인자) (https://sirzzang.github.io/dev/Dev-python-super/)
```

## 2. 분류기 모델 설계
- CNN (Convolutional Neural Network)를 사용하여 모델 설계합니다.
- 이미지는 3-channel을 입력으로 받고, 최종적으로 10개의 class에 대한 확률을 구합니다.
- 아래의 모델을 예시이므로, layer를 직접 추가해보면서 실험하시길 바랍니다.

### Classifier 클래스
- nn.Module: 뉴럴넷 구현을 위한 base class. forward, parameter 등 모델을 만들고 사용할 때 필요한 부분들이 내부적으로 구현되어 있음.
- torch.nn은 모델을 정의할 때 사용하는 Class들을 포함하고 있음.

- init: python class의 constructor. 필요한 멤버변수들을 초기화하고, nn.Sequential or nn.ModuleList를 이용하여 모델의 구성을 정의한다.

- forward: 모델의 input을 받고 output을 return하는 함수.

### Pytorch functions
- ``nn.Conv2d`` : torch.nn.Conv2d(in_channels: int, out_channels: int, kernel_size, stride = 1, padding = 0)
- ``nn.Maxpool2d`` : torch.nn.MaxPool2d(kernel_size, stride = None, padding = 0)
- ``nn.Linear`` : torch.nn.Linear(in_features: int, out_features: int, bias: bool = True)


**references**

https://tutorials.pytorch.kr/beginner/examples_nn/two_layer_net_module.html

https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

https://pytorch.org/docs/stable/generated/torch.nn.Linear.html


**tips**

http://taewan.kim/post/cnn/

http://taewan.kim/post/cnn/#4-cnn-%EC%9E%85%EC%B6%9C%EB%A0%A5-%ED%8C%8C%EB%A6%AC%EB%AF%B8%ED%84%B0-%EA%B3%84%EC%82%B0

https://mrsyee.github.io/image%20processing/2018/11/28/cnn_technique/


In [15]:
import torch.nn as nn
import torch.nn.functional as F

# Req. 1-2	분류기 모델 설계
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()       # CNN 클래스를 상속받겠다. 기존 이미지에서 CNN을 통해 feature extraction 한 후에, FC에 넣는 것이 특징적임
        
        # 이미지 3-channel 입력
        self.conv1 = nn.Conv2d(           # 이미지를 직접 넣는다
            in_channels = 3,              # RGB이므로 3
            out_channels = 8,             # 한 칸을 필터가 몇번 훑느냐
            kernel_size = 3,              # 필터의 크기
            padding = 1)                  # 이미지 가는 중앙에 비해 덜 연산되는 것을 막기 위함. 숫자 맞추는 거 어려움.
        self.conv2 = nn.Conv2d(
            in_channels = 8,              # 이전 레이어의 output이 8-depth 이므로
            out_channels = 16,            
            kernel_size = 3,
            padding = 1
        )
        self.pool = nn.MaxPool2d(
            kernel_size = 2,              # pooling하는 필터의 크기
            stride = 2)
                                          # 이 과정을 거쳐 탄생한 feature map은 합해지고 겹쳐지는 과정에서 주변 정보를 반영한다.
        # 최종 10개의 class에 대한 확률
        self.fc1 = nn.Linear(8*8*16, 64)  # input 숫자는 계산해서 나와야 함
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)      # 최종적으로 10개의 클래스로 분류하므로
    

    def forward(self, x):
        x = self.conv1(x)                 
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8*8*16)            # FC layer는 1차원을 입력받는다. 2차원 데이터를 1차원 데이터로 변환한다.
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x)              # softmax는 각 클래스에 속할 확률을 나타내며, 총합은 1이다.

        return x

classifier = Classifier()

## 3. Loss function 정의
- 분류기를 위해서 cross-entropy loss를 사용합니다.
- optimzer로 SGD를 사용하고, learning rate, momentum등을 설정해줍니다.

### Pytorch functions
- ``nn.CrossEntropyLoss`` : torch.nn.CrossEntropyLoss(weight: Optional[torch.Tensor] = None, size_average=None)
- ``optim.SGD`` :torch.optim.SGD(params, lr=<required parameter>, momentum=0)

**references**

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

https://pytorch.org/docs/stable/optim.html

**tips**

https://forensics.tistory.com/28

https://wordbe.tistory.com/entry/ML-Cross-entropyCategorical-Binary%EC%9D%98-%EC%9D%B4%ED%95%B4

https://gomguard.tistory.com/187


In [16]:
import torch.optim as optim

# Req. 1-3	Loss function 및 optimizer정의
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)


## 4. 모델 학습시키기
1) **첫번째 for문**으로 epochs 만큼 반복

2) **두번째 for문**으로 trainset이 저장되어 있는 **trainloader**에서 배치 사이즈 만큼씩 샘플링하여 data load

3) load한 data에서 input 값과 label로 분리하여 저장

4) 각각의 값을 device에 올린다 (GPU or CPU)

5) optimizer에서 gradient 값 0으로 초기화

6) model에 input값을 입력하여 forward propagation

7) loss function으로 예측값과 label 비교

8) loss 값 backpropagation 하여 gradient 계산

9) 계산된 gradient를 모두 parameter에 적용

10) loss 값을 합하여 일정 주기(ex.2000 batch) 마다 평균 loss 값 출력 후 초기화

11) 2)로 돌아가 반복 한뒤 2)가 모두 마치면 1)로 돌아가 반복

12) 학습이 마친 이후 모델 저장

### Pytorch functions
- ``nn.CrossEntropyLoss`` : torch.nn.CrossEntropyLoss(weight: Optional[torch.Tensor] = None, size_average=None)
- ``optim.SGD`` :torch.optim.SGD(params, lr=<required parameter>, momentum=0)

**references**

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

https://pytorch.org/docs/stable/optim.html

https://tutorials.pytorch.kr/beginner/saving_loading_models.html

**tips**

https://forensics.tistory.com/28

https://wordbe.tistory.com/entry/ML-Cross-entropyCategorical-Binary%EC%9D%98-%EC%9D%B4%ED%95%B4

https://gomguard.tistory.com/187


### GPU 
GPU가 있다면 GPU를 통해 학습을 가속화하고, 없으면 CPU로 학습하기 위해 device를 정해줍니다.


In [17]:
# Req. 1-4	모델 학습
# model을 device에 올린다 (GPU or CPU) 
# 구현 완료 상태
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classifier = classifier.to(DEVICE)

In [18]:
# Req. 1-4	모델 학습
epochs = 10 #define epochs

# 1) for문으로 epochs 만큼 반복
for epoch in range(epochs):  # epochs 횟수만큼 반복
    
    # loss값 누적 
    running_loss = 0.0
    
    # 2) for문으로 trainset이 저장되어 있는 trainloader에서 배치 사이즈 만큼씩 샘플링하여 data load
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        # 3) load한 data에서 input 값과 label로 분리하여 저장
       
        # 4) 각각의 값을 device에 올린다 (GPU or CPU)
        inputs = inputs.to(DEVICE)
        labels = labels.to(device=DEVICE, dtype=torch.int64)


        # 5) optimizer에서 gradient 값 0으로 초기화
        optimizer.zero_grad()

        # 6) model에 input값을 입력하여 forward propagation
        outputs = classifier(inputs)


        # 7)  loss function으로 예측값과 label 비교
        loss = criterion(outputs, labels)
        
        # 8) loss 값 backpropagation 하여 gradient 계산
        loss.backward()
        
        # 9) 계산된 gradient를 모두 parameter에 적용
        optimizer.step()

        # 10) loss 값을 합하여 일정 주기(ex.2000 batch) 마다 평균 loss 값 출력 후 초기화
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

# 12) torch.save로 학습이 마친 이후 모델 저장        



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   200] loss: 2.305
[1,   400] loss: 2.299
[1,   600] loss: 2.289
[1,   800] loss: 2.265
[1,  1000] loss: 2.221
[1,  1200] loss: 2.176
[1,  1400] loss: 2.100
[2,   200] loss: 1.956
[2,   400] loss: 1.900
[2,   600] loss: 1.842
[2,   800] loss: 1.794
[2,  1000] loss: 1.763
[2,  1200] loss: 1.716
[2,  1400] loss: 1.693
[3,   200] loss: 1.619
[3,   400] loss: 1.621
[3,   600] loss: 1.596
[3,   800] loss: 1.555
[3,  1000] loss: 1.543
[3,  1200] loss: 1.540
[3,  1400] loss: 1.527
[4,   200] loss: 1.469
[4,   400] loss: 1.446
[4,   600] loss: 1.402
[4,   800] loss: 1.434
[4,  1000] loss: 1.391
[4,  1200] loss: 1.386
[4,  1400] loss: 1.393
[5,   200] loss: 1.354
[5,   400] loss: 1.342
[5,   600] loss: 1.320
[5,   800] loss: 1.300
[5,  1000] loss: 1.286
[5,  1200] loss: 1.275
[5,  1400] loss: 1.289
[6,   200] loss: 1.254
[6,   400] loss: 1.251
[6,   600] loss: 1.255
[6,   800] loss: 1.235
[6,  1000] loss: 1.248
[6,  1200] loss: 1.224
[6,  1400] loss: 1.214
[7,   200] loss: 1.187
[7,   400] 

In [19]:
# 12) torch.save로 학습이 마친 이후 모델 저장  
PATH = './cifar_classifier2.pth'
torch.save(classifier.state_dict(), PATH)

print('Finished Training')

Finished Training


## 5. 테스트하기 
- 학습이 완료된 모델을 testset이 저장되어 있는 **testloader**에 대하여 성능 평가
- 모델을 evaluation 모드로 변경
- gradient계산 할 필요가 없고, backpropagation도 진행하지 않는다.

### 진행 방식
만약 저장한 모델을 load해야 한다면, 모델의 인스턴스를 생성하고, 모델의 weight이 저장되어 있는 .ckpt 파일을 모델에 load

1) 모델을 evaluation 모드로 전환

2) with torch.no_grad로 gradient 계산을 제외

3) for문으로 testset에 저장되어 있는 testloader에서 배치 사이즈 만큼씩 샘플링하여 data load

4) load한 data에서 input 값과 label로 분리하여 저장

5) 각각의 값을 device에 올린다 (GPU or CPU)

6) model에 input값을 입력하여 forward propagation

7) 예측한 값들 중 가장 높은 확률의 class 선택

8) label과 예측한 class 비교하여 정답 확인

9) 정답률 출력

### Pytorch functions
- ``torch.no_grad`` : gradient를 계산하기 위해 추적하는 수고를 하지 않음



In [23]:
# Req. 1-5	모델 테스트
# 만약 저장한 모델을 load해야 한다면, 모델의 인스턴스를 생성하고, 모델의 weight이 저장되어 있는 .ckpt 파일을 모델에 load
new_classifier = Classifier()
new_classifier.load_state_dict(torch.load('./cifar_classifier2.pth'))
new_classifier.to(DEVICE)

Classifier(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [24]:

# 1) 모델을 evaluation 모드로 전환
new_classifier.eval()
correct = 0
total = 0
# 2) with torch.no_grad로 gradient 계산을 제외
with torch.no_grad():
    # 3) for문으로 testset에 저장되어 있는 testloader에서 배치 사이즈 만큼씩 샘플링하여 data load
    for data in testloader:
        
         # 4) load한 data에서 input 값과 label로 분리하여 저장
        inputs, labels = data
       
        # 5) 각각의 값을 device에 올린다 (GPU or CPU)
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        # 6) model에 input값을 입력하여 forward propagation
        outputs = new_classifier(inputs)

        # 7) 예측한 값들 중 가장 높은 확률의 class 선택
        _, predicted = torch.max(outputs.data, 1)


        # 8) label과 예측한 class 비교하여 정답 확인
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 9) 정답률 출력
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the 10000 test images: 61 %
